In [11]:
#TODOs 
# Add Data Origin (Linkedin in this case)

In [12]:
from urllib.request import urlopen as ureq
import urllib

In [13]:
import pandas as pd
import re
import random
from datetime import date, timedelta

import selenium
from bs4 import BeautifulSoup as soup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [14]:
from bs4 import BeautifulSoup as soup

# Scraping AU

In [15]:
url = 'https://www.glassdoor.com/Job/sydney-data-analyst-jobs-SRCH_IL.0,6_IC2235932_KO7,19.htm?fromAge=1&radius=50&sortBy=date_desc'

### Opening up connection and getting HTML info from the webpage

In [16]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [17]:
# New jobs posted within set timeframe
new_jobs_posted = int(re.match('\d+',driver.find_element_by_xpath('//h1[@data-test="jobCount-H1title"]').text).group(0))
new_jobs_posted

33

In [18]:
# Finding and saving the current page and the total number of pages to loop through
current_page = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[0])
total_number_of_pages = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[1])
print(current_page)
total_number_of_pages

1


2

In [19]:
#Instatiating the lists to fill when looping
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter1=0
counter2=0


#Check for the app to stop if the job was already scrapped
try:
    df = pd.read_csv('Glassdoor_data_info_au.csv', engine='python')
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
except:
    pass

In [20]:
contenedores = []


while current_page <= total_number_of_pages:
    contenedores = driver.find_elements_by_xpath('.//li[@data-adv-type="GENERAL"]')
    try:
        driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
    except:
        pass
    if counter1 < len(contenedores):
        for job_post in contenedores:
            job_titles.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').text)
            company_name.append(job_post.find_element_by_xpath('.//div[@class="d-flex justify-content-between align-items-start"]').text)
            job_location.append(job_post.find_element_by_xpath('.//span[@class="pr-xxsm css-1ndif2q e1rrn5ka0"]').text)
            date_scraped.append(str(date.today()))
            links.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').get_attribute('href'))
            counter1 = counter1 + 1    


    for i in range(-1,len(contenedores)):
        driver.find_elements_by_xpath('//li[@data-adv-type="GENERAL"]')[i].click()
        sleep(random.uniform(2.1,5.6))
        try:
            driver.find_element_by_xpath('//span[@alt="Close"]').click()
        except:
            driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb5"]').click()
            job_desc.append(driver.find_element_by_xpath('//div[@class="jobDescriptionContent desc"]').text)
            #Scrolling sidebar inside side-container
            try:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
            except:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6"]'), -7000)
            sleep(random.uniform(1.4,3.8))
            job_industry.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[3]').text)
            job_type.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[1]').text)
            job_seniority.append('NA')
            counter2 = counter2 + 1
            if counter2 >= len(contenedores):
                break  
    if counter2 >= len(contenedores):
        break  
        
    

In [21]:
print('Job titles ----')
print(len(job_titles))    
print(job_titles[0:2])
print('Job company ----')
print(len(company_name))
print(company_name[0:2])
print('Job Location ----')
print(len(job_location))
print(job_location[0:2])
print('Date scraped ----')
print(len(date_scraped))
print(date_scraped[0:2])
print('Job links ----')
print(len(links))
print(links[0:2])
print('Job type ----')
print(len(job_type))
print(job_type[0:2])
print('Job industry ----')
print(job_industry[0:2])
print('Job seniority ----')
print(job_seniority[0:2])

Job titles ----
30
['Data Analyst', 'Data Analyst- Financial Crimes']
Job company ----
30
['BizCover', 'AMP Limited']
Job Location ----
30
['Sydney', 'Sydney']
Date scraped ----
30
['2021-08-25', '2021-08-25']
Job links ----
30
['https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=1136043&s=58&guid=0000017b7d7b4811a06c3b6b4f6e8c2a&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_02d1d641&cb=1629897836919&jobListingId=1007258570734&jrtk=2-0-1fdunmi1qu4m6801-1fdunmi2lu2bn800-35261cc787728611', 'https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=1136043&s=58&guid=0000017b7d7b4811a06c3b6b4f6e8c2a&src=GD_JOB_AD&t=SR&vt=w&cs=1_d158d5d8&cb=1629897836919&jobListingId=1007258986073&jrtk=2-0-1fdunmi1qu4m6801-1fdunmi2lu2bn800-6faa606397ff95bf']
Job type ----
30
['Job Type : Full-time', 'Job Type : Full-time']
Job industry ----
['Industry : Insurance', 'Industry : Finance']
Job seniority ----
['NA', 'NA']


In [22]:
print(job_titles[2])
print(company_name[2])
print(job_location[2])
print(date_scraped[2])
print(links[2])
print(job_type[2])
print(job_industry[2])
print(job_seniority[2])

Data Analyst
Preacta Recruitment
Sydney
2021-08-25
https://www.glassdoor.com/partner/jobListing.htm?pos=103&ao=1136043&s=58&guid=0000017b7d7b4811a06c3b6b4f6e8c2a&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_3319a5f9&cb=1629897836920&jobListingId=1007258665399&jrtk=2-0-1fdunmi1qu4m6801-1fdunmi2lu2bn800-6c97a4e6153cb362
Job Type : N/A
Industry : N/A
NA


In [23]:
print(job_titles[1])
print(company_name[1])
print(job_location[1])
print(date_scraped[1])
print(links[1])
print(job_type[1])
print(job_industry[1])
print(job_seniority[1])

Data Analyst- Financial Crimes
AMP Limited
Sydney
2021-08-25
https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=1136043&s=58&guid=0000017b7d7b4811a06c3b6b4f6e8c2a&src=GD_JOB_AD&t=SR&vt=w&cs=1_d158d5d8&cb=1629897836919&jobListingId=1007258986073&jrtk=2-0-1fdunmi1qu4m6801-1fdunmi2lu2bn800-6faa606397ff95bf
Job Type : Full-time
Industry : Finance
NA


In [24]:
job_titles

['Data Analyst',
 'Data Analyst- Financial Crimes',
 'Data Analyst',
 'Project Data Analyst',
 'Senior Data Analyst',
 'Quality Analyst Tester',
 'Business analyst Regulatory reporting',
 'KYC Operations – Client Onboarding Analyst - Associate',
 'Data Reporting Analyst- 12 month fixed term',
 'Credit Risk Analyst - Sydney',
 'Data Analyst (12 mth FTC, SYD based)',
 'Data Assurance Specialists - Tech Consulting, Data & Analytics',
 'IT Risk & Controls Analyst',
 'Test Analyst',
 'Data Analyst (Consultant/Senior Consultant), Quantitative Services, Tax',
 'Treasury / Risk Analyst',
 'Business Systems Analyst',
 'Pricing Analyst - 12 Month FTC',
 'Senior Actuarial Analyst',
 'Senior Consultant, Data & Analytics, Technology Consulting',
 'Risk Surveillance Frameworks and Operations Senior Analyst',
 'Range Analyst',
 'Data Engineer - TM1/Planning Analytics',
 'Business Analyst',
 'Consultant / Senior Consultant, Tech Business Analyst - Data Risk & Controls, Technology Consulting',
 'Senior

In [25]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [26]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1 = 0

df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [27]:
df

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
373,Data Analyst,BizCover,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Insurance,Job Type : Full-time,NA,Join a supportive and growing team!\nTake owne...
374,Data Analyst- Financial Crimes,AMP Limited,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NA,Working for AMP means being part of a company ...
375,Data Analyst,Preacta Recruitment,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : N/A,NA,A reputable organisation is looking for a Data...
376,Project Data Analyst,Northbridge Recruitment,,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : N/A,NA,Work in a tier 1 company\nFlexible and diverse...
377,Senior Data Analyst,IOOF Holdings,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NA,"Date: 25-Aug-2021\nLocation: Sydney, AU, 2000\..."
378,Quality Analyst Tester,Manvision Consulting Pvt Ltd,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Business Services,Job Type : Full-time,NA,Manvision Consulting provides IT Contracting a...
379,Business analyst Regulatory reporting,Tekskills pty ltd,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NA,Must have regulatory reporting experience.\nCo...
380,KYC Operations – Client Onboarding Analyst - A...,Deutsche Bank,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Finance,Job Type : Full-time,NA,Position Overview\nDetails of the Division and...
381,Data Reporting Analyst- 12 month fixed term,Genesis IT&T Pty Ltd,City of Parramatta,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Business Services,Job Type : N/A,NA,An ASX listed company is actively recruiting a...
382,Credit Risk Analyst - Sydney,Atradius,Sydney,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Insurance,Job Type : Full-time,NA,Job Description\n\nAtradius Australia is looki...


In [28]:
df.to_csv('Glassdoor_data_info_au.csv',mode='a',header=False)

# Scraping US

In [29]:
url = 'https://www.glassdoor.com/Job/us-data-analyst-jobs-SRCH_IL.0,2_IN1_KO3,15.htm?fromAge=1&sortBy=date_desc'

### Opening up connection and getting HTML info from the webpage

In [30]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [31]:
# New jobs posted within set timeframe
new_jobs_posted = int(re.match('\d+',driver.find_element_by_xpath('//h1[@data-test="jobCount-H1title"]').text).group(0))
new_jobs_posted

439

In [32]:
# Finding and saving the current page and the total number of pages to loop through
current_page = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[0])
total_number_of_pages = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[1])
print(current_page)
total_number_of_pages

1


15

In [33]:
#Instatiating the lists to fill when looping
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter1=0
counter2=0


#Check for the app to stop if the job was already scrapped
try:
    df = pd.read_csv('Glassdoor_data_info_us.csv', engine='python')
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
except:
    last_index_in_csv = 0

In [34]:
last_index_in_csv

248

In [35]:
contenedores = []


while current_page <= total_number_of_pages:
    contenedores = driver.find_elements_by_xpath('.//li[@data-adv-type="GENERAL"]')
    try:
        driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
    except:
        pass
    if counter1 < len(contenedores):
        for job_post in contenedores:
            job_titles.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').text)
            company_name.append(job_post.find_element_by_xpath('.//div[@class="d-flex justify-content-between align-items-start"]').text)
            job_location.append(job_post.find_element_by_xpath('.//span[@class="pr-xxsm css-1ndif2q e1rrn5ka0"]').text)
            date_scraped.append(str(date.today()))
            links.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').get_attribute('href'))
            counter1 = counter1 + 1    


    for i in range(-1,len(contenedores)):
        driver.find_elements_by_xpath('//li[@data-adv-type="GENERAL"]')[i].click()
        sleep(random.uniform(2.1,5.6))
        try:
            driver.find_element_by_xpath('//span[@alt="Close"]').click()
        except:
            driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb5"]').click()
            job_desc.append(driver.find_element_by_xpath('//div[@class="jobDescriptionContent desc"]').text)
            #Scrolling sidebar inside side-container
            try:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
            except:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6"]'), -7000)
            sleep(random.uniform(1.4,3.8))
            job_industry.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[3]').text)
            job_type.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[1]').text)
            job_seniority.append('NA')
            counter2 = counter2 + 1
            if counter2 >= len(contenedores):
                break  
    if counter2 >= len(contenedores):
        break  
        
    

In [36]:
driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb5"]').text

'Show Less'

In [37]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [38]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1 = 0
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [39]:
df

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
249,Data Analyst,ParkMobile,"Atlanta, GA",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,To Power Smart Mobility for Every Driver and V...
250,"Data Analyst, Ads Finance",Google,"New York, NY",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,Note: By applying to this position you will ha...
251,Data Analyst,"S&H Industrial Services, Inc","Franklin, TN",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NA,Description:\n\nS&H Systems is a single-source...
252,IT Data Analyst,Chadwell Supply,"Tampa, FL",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Transportation & Logistics,Job Type : Full-time,NA,"Description:\nIT DATA ANALYST\n\nSmart, Innova..."
253,Data Analyst,Volanno,Remote,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,Description:\n\nThe Data Analyst is responsibl...
254,Finance & Data Analyst,GivePower,Remote,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NA,WHO WE ARE\nThe GivePower Foundation (“GivePow...
255,Data Analyst,AE Stategies,"Washington, DC",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : N/A,Job Type : Full-time,NA,Description:\nAbout Us\nAE Strategies is a Mcl...
256,Data Analyst Intern (m/f/d),Nordex SE,"West Branch, IA",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Manufacturing,Job Type : Internship,NA,ABOUT THE NORDEX GROUP\nThe manufacture of win...
257,Associate Data Analyst - Primary Care,Mayo Clinic,"Eau Claire, WI",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Health Care,Job Type : Full-time,NA,City\nEau Claire\nState\nWisconsin\nTelecommut...
258,Data Analyst,Facebook,"Burlingame, CA",2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,"At Facebook Reality Labs (FRL), our goal is to..."


In [40]:
df.to_csv('Glassdoor_data_info_us.csv',mode='a',header=False)

# Scraping SP

In [41]:
url = 'https://www.glassdoor.com/Job/spain-data-analyst-jobs-SRCH_IL.0,5_IN219_KO6,18.htm?fromAge=1&includeNoSalaryJobs=true'

### Opening up connection and getting HTML info from the webpage

In [42]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [43]:
# New jobs posted within set timeframe
new_jobs_posted = int(re.match('\d+',driver.find_element_by_xpath('//h1[@data-test="jobCount-H1title"]').text).group(0))
new_jobs_posted

18

In [44]:
# Finding and saving the current page and the total number of pages to loop through
current_page = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[0])
total_number_of_pages = int(re.findall('\d+',driver.find_element_by_xpath('//div[@class="cell middle d-none d-md-block py-sm"]').text)[1])
print(current_page)
total_number_of_pages

1


1

In [45]:
#Instatiating the lists to fill when looping
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter1=0
counter2=0


#Check for the app to stop if the job was already scrapped
try:
    df = pd.read_csv('Glassdoor_data_info_sp.csv', engine='python')
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
except:
    last_index_in_csv = 0

In [46]:
last_index_in_csv

158

In [47]:
contenedores = []


while current_page <= total_number_of_pages:
    contenedores = driver.find_elements_by_xpath('.//li[@data-adv-type="GENERAL"]')
    try:
        driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
    except:
        pass
    if counter1 < len(contenedores):
        for job_post in contenedores:
            job_titles.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').text)
            company_name.append(job_post.find_element_by_xpath('.//div[@class="d-flex justify-content-between align-items-start"]').text)
            job_location.append(job_post.find_element_by_xpath('.//span[@class="pr-xxsm css-1ndif2q e1rrn5ka0"]').text)
            date_scraped.append(str(date.today()))
            links.append(job_post.find_element_by_xpath('.//a[@data-test="job-link"]').get_attribute('href'))
            counter1 = counter1 + 1    


    for i in range(-1,len(contenedores)):
        driver.find_elements_by_xpath('//li[@data-adv-type="GENERAL"]')[i].click()
        sleep(random.uniform(2.1,5.6))
        try:
            driver.find_element_by_xpath('//span[@alt="Close"]').click()
        except:
            driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb5"]').click()
            job_desc.append(driver.find_element_by_xpath('//div[@class="jobDescriptionContent desc"]').text)
            #Scrolling sidebar inside side-container
            try:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6 opened transformNone"]'), -7000)
            except:
                driver.execute_script("arguments[0].scrollTop = arguments[1]", driver.find_element_by_xpath('//div[@class="p-0 css-rfn2ib exy0tjh6"]'), -7000)
            sleep(random.uniform(1.4,3.8))
            job_industry.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[3]').text)
            job_type.append(driver.find_element_by_xpath('//div[@class="css-bhqyka epgue5a3"][2]/div[1]').text)
            job_seniority.append('NA')
            counter2 = counter2 + 1
            if counter2 >= len(contenedores):
                break  
    if counter2 >= len(contenedores):
        break  
        
    

In [48]:
driver.find_element_by_xpath('//div[@class="css-t3xrds e856ufb5"]').text

'Show Less'

In [49]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [50]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1 = 0
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [51]:
df

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
159,Talent Development Analyst,Cabify,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : N/A,NA,Sobre el puesto\nDo you want to help us to cha...
160,Project Data Analyst,CHEP,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Transportation & Logistics,Job Type : Full-time,NA,"Las empresas del Grupo Brambles, entre ellas C..."
161,Junior Media Analyst,Fever,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,We’re Fever!\nFever is the leading global ente...
162,"Continuous Improvement, Lean and Quality Analyst",Chubb INA Holdings Inc.,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Insurance,Job Type : Full-time,NA,The role incumbent is responsible for identify...
163,Junior Data Scientist (f/m),Axpo,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,"Industry : Oil, Gas, Energy & Utilities",Job Type : Part-time,NA,Junior Data Scientist (f/m)\n100% | Madrid\nTh...
164,Project Data Analyst,,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Transportation & Logistics,Job Type : Full-time,NA,"Las empresas del Grupo Brambles, entre ellas C..."
165,Senior Data Scientist (f/m),Axpo,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,"Industry : Oil, Gas, Energy & Utilities",Job Type : Part-time,NA,Senior Data Scientist (f/m)\n100% | Madrid\nTh...
166,Data Engineer (also available for full remote ...,lastminute.com,Madrid,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Travel & Tourism,Job Type : Full-time,NA,"Company Description\n\nLaunched in 1998, this ..."
167,Business Analyst (m/f/d),TUI,Arca,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Travel & Tourism,Job Type : N/A,NA,For TUI Group IT we are looking for a Business...
168,FP&A Analyst,Criteo,Spain,2021-08-25,https://www.glassdoor.com/partner/jobListing.h...,Industry : Information Technology,Job Type : Full-time,NA,"Who we are\nAt Criteo, our culture is as uniqu..."


In [52]:
df.to_csv('Glassdoor_data_info_sp.csv',mode='a',header=False)